# This notebook requires a deployment named "gpt-4o", edit as needed

## Web Search Ant As Tool

This example demonstrates one ant using another ant as a tool so that the second ant's work is outside the main thread.

The **WebSearchAgentAnt** demonstrates the use of a Web API secured using an API key with an auth header. 
This sample requires you to create a Web Search (bing) resource in Azure and enter its corresponding API key in [0-AI-settings](0-AI-settings.ipynb)

WebSearchAgentAnt uses **WebPageAgentAnt** via a tool call to evaluate search results using the search results link.

This demo uses a lot of tokens. 

**If your quota is low, you will get throttled**

## First steps
- First, download AntRunLib from Nuget
- At least once, set up the environment using **[0-AI-settings](0-AI-settings.ipynb)**

In [8]:
#r "nuget: AntRunnerLib, 0.8.6"

using AntRunnerLib;
using AntRunnerLib.Identity;
using static AntRunnerLib.ClientUtility;
using System.IO;

#!import config/Settings.cs 

var envVariables = Settings.GetEnvironmentVariables();
foreach (var kvp in envVariables)
{
    Environment.SetEnvironmentVariable(kvp.Key, kvp.Value);
}

Environment.SetEnvironmentVariable("AZURE_OPENAI_DEPLOYMENT", "gpt-4o");

var config = AzureOpenAiConfigFactory.Get();
var client = GetOpenAiClient(config);


Installed Packages AntRunnerLib, 0.8.6

In [9]:
var assistantId = await AssistantUtility.Create("WebSearchAgentAnt", config);
Console.WriteLine(assistantId);
assistantId = await AssistantUtility.Create("WebPageAgentAnt", config);
Console.WriteLine(assistantId);

asst_QcQ2MZ6Ri8padiBwOYbUaZ6u
asst_9n6NTEkCykG9W6mruahiHsKz


In [10]:
AntLoader.LoadAssembly(@".\AssistantDefinitions\WebPageAgentAnt\HtmlAgilityPack.dll");
AntLoader.LoadAssembly(@".\AssistantDefinitions\WebPageAgentAnt\HtmlAgilityPackExtensions.dll");

## Run the Assistant

`output.Dialog` shows the conversation.

In [11]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "WebSearchAgentAnt",
    Instructions = "Use python to get todays date and then use it to find things to do in Atlanta today. Be careful to not include events in the past in your final answer.",
};
var output = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
output.Dialog.DisplayAs("text/markdown");


User: Use python to get todays date and then use it to find things to do in Atlanta today. Be careful to not include events in the past in your final answer.

Assistant: I ran this code:
```
from datetime import datetime

# Get today's date
today = datetime.today().strftime('%Y-%m-%d')
today
```

I called the tool named crawl with these arguments:
```
{"q": "things to do in Atlanta on 2024-09-25", "answerCount": 5, "textFormat": "HTML"}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta on 2024-09-25"},"webPages":{"value":[{"name":"Atlanta Events September 2024: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/september/","displayUrl":"https://atlanta-ga.events/september","snippet":"This guide is your gateway to discovering the best things to do in Atlanta, from Concerts and Theater Shows to Sports events and music festivals. You\u2019ll find a detailed list of all the most engaging and exciting activities happening throughout the month. ... Sep 25, 2024 6:30 PM . Lorna Shore . Upcoming Events: 2. The Tabernacle - GA . Sep 25 ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002B2024-09-25\u0026d=4854189676169673\u0026mkt=en-US\u0026setlang=en-US\u0026w=73iRaVHIvJefbkmdL1lCvH4mS1bjztOY"},{"name":"Events and Things to Do in Atlanta GA this from 2024-09-28 to 2024-09-29","url":"https://eventcartel.com/events/all-categories/atlanta/from_2024-09-28_to_2024-09-29","displayUrl":"https://eventcartel.com/events/all-categories/atlanta/from_2024-09-28_to_2024-09-29","snippet":"Explore a wide range of events and things to do from 2024-09-28 to 2024-09-29 tailored to offer you the best in entertainment and fun things to do in Atlanta. Whether you\u2019re interested in the latest concerts, theater shows, or other local events happening from 2024-09-28 to 2024-09-29, everything you need is just a few clicks away.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002B2024-09-25\u0026d=593829212581\u0026mkt=en-US\u0026setlang=en-US\u0026w=ArLA-810ZZkaWFgm_VL8k_3YAMhRPChX","thumbnailUrl":"https://www.bing.com/th?id=OIF.j9DT03xNjIPyoPYsyjOF4w\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Cool Things To Do in September in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta","snippet":"The Sweet Auburn Music Fest, which takes place Sept. 28-29. The festival, located on Auburn Avenue, one of Atlanta\u2019s most iconic streets, offers fun-filled entertainment, live music \u2014 everything from R\u0026amp;B to gospel to alternative favorites \u2014 international food court, national artists, celebrities and vendors of all types.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002B2024-09-25\u0026d=4692574352310340\u0026mkt=en-US\u0026setlang=en-US\u0026w=c-5Oucqqe-FV74GkIkdoRE4k4vk9MNbJ","thumbnailUrl":"https://www.bing.com/th?id=OIP.m0ENCio7ytS2o9PQRWDCRgHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Events - September 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html","snippet":"More details. \u2013 Dragon Con, August 29 - September 2 2024. One of the biggest annual pop culture conventions in the country. More details. \u2013 Atlanta Black Theatre Festival, August 30 - September 1 2024. Annual festival celebrating the Black experience in the performing arts, with plays, panels, masterclasses and other events.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002B2024-09-25\u0026d=4712253893574850\u0026mkt=en-US\u0026setlang=en-US\u0026w=aWkJWqw243Je4oOvzq1dGpeg_GE4sHjt","thumbnailUrl":"https://www.bing.com/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Explore Atlanta Events for Today, This Week or Weekend","url":"https://discoveratlanta.com/events/main/","displayUrl":"https://discoveratlanta.com/events/m","snippet":"Explore the Atlanta events calendar for things to do today, this week or this weekend. Find upcoming festivals, seasonal events and more! ... Every Saturday until 09/28/2024. Protect Yo Energy Con 9/24/24 \u2013 11/9/24. ... Every Wednesday until 09/25/2024","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002B2024-09-25\u0026d=4819881473496114\u0026mkt=en-US\u0026setlang=en-US\u0026w=zvJBiG0Dg4eP1GMx_bJv9FTM8ziERJGY","thumbnailUrl":"https://www.bing.com/th?id=OIP.FkxSV6JbrWatsFMM3_8dIAHaEh\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Upcoming Events - Discover Atlanta","url":"https://discoveratlanta.com/events/all/","displayUrl":"https://discoveratlanta.com/events/all","snippet":"Bounce through the incredible events happening around Atlanta today, tomorrow, this week or this month. Plan your fun and be in the know on everything from family-favorites, to can\u2019t-miss hot spots, concerts and foodie festivals. Showing 1 \u2013 15 of 495. Editors\u2019 Pick.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002B2024-09-25\u0026d=4829094184573839\u0026mkt=en-US\u0026setlang=en-US\u0026w=J3F_lNIMaK2UBKZlUeD3eG4MmVcBwK29"},{"name":"September in Atlanta: Top things to do this month","url":"https://www.atlantaonthecheap.com/september-things-to-do-atlanta/","displayUrl":"https://www.atlantaonthecheap.com/september-things-to-do-atlanta","snippet":"Sunday, September 15, 2024 During Streets Alive, for four glorious hours, miles of Atlanta roads are closed to cars \u2014 and open for people to stroll, roll, bike, and play! This fall, the route will stretch from Atlanta\u2019s West End to Grant Park \u2014 connecting seven southwest neighborhoods via Ralph David Abernathy Blvd. and Georgia Ave.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002B2024-09-25\u0026d=4531508777067252\u0026mkt=en-US\u0026setlang=en-US\u0026w=dBvGHC3HqGEMsXsP8SfKEFE-tgkhrh_1","thumbnailUrl":"https://www.bing.com/th?id=OIP.87SMQKoDZPtIKmUHIBiCMQHaFj\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"35\u002B Best Things to Do in Atlanta in 2024 - Unexpected Atlanta","url":"https://unexpectedatlanta.com/best-things-to-do-in-atlanta/","displayUrl":"https://unexpectedatlanta.com/best-things-to-do-in-atlanta","snippet":"1. Visit the National Parks on a Fee-Free Day. First on our list of the best things to do in Atlanta in 2024, check out America\u2019s National Park System\u2019s fee-free days this year. On these special days, you can visit any of the United States\u2019 400\u002B National Parks for free, including places in and around Georgia.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002B2024-09-25\u0026d=4885792044096348\u0026mkt=en-US\u0026setlang=en-US\u0026w=SRiegEmhRGKtzRRUG028yHS95G2gOIKx","thumbnailUrl":"https://www.bing.com/th?id=OIP.HwzfdFE6-tSHfhoFlTxXKQHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Things to Do in Atlanta in 2024: Events, festivals, and exhibits ...","url":"https://www.thrillist.com/events/atlanta/events-calendar-in-atlanta-this-year","displayUrl":"https://www.thrillist.com/events/atlanta/events-calendar-in-atlanta-this-year","snippet":"August 23\u201325, 2024. The Fox Theatre. Every August, the Fox Theatre pays homage to its history as an iconic movie venue with its annual Coca Cola Summer Film Festival. This year\u2019s edition ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.WUHCPl7_RpepE3pSEdAEMgHaE8\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"THE 15 BEST Things to Do in Atlanta (2024) - Tripadvisor","url":"https://www.tripadvisor.com/Attractions-g60898-Activities-zfr7917665-Atlanta_Georgia.html","displayUrl":"https://www.tripadvisor.com/Attractions-g60898-Activities-zfr7917665-Atlanta_Georgia.html","snippet":"2024. 2. Atlanta Botanical Garden. 5,236. Gardens. Step into a world of magic and serenity at the Atlanta Botanical Garden, where features like the Fuqua Orchid Center treat visitors to a rare collection of high-elevation orchids never before grown\u2026. See tours. 3. World of Coca-Cola.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002B2024-09-25\u0026d=4999832014161720\u0026mkt=en-US\u0026setlang=en-US\u0026w=GtUyw72N_XrhqWtUQWQStMnnppmx6J8T"}]}}
```

I called the tool named crawl with these arguments:
```
{"q": "events in Atlanta on September 25, 2024", "answerCount": 5, "textFormat": "HTML"}
```
and got this result:
```
{"queryContext":{"originalQuery":"events in Atlanta on September 25, 2024"},"webPages":{"value":[{"name":"Atlanta Events September 2024: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/september/","displayUrl":"https://atlanta-ga.events/september","snippet":"Check Out The Atlanta Events Calendar for September 2024. See the List Of All Current \u0026amp; Upcoming Events at the Lowest Possible Price. ... Sep 25, 2024 6:30 PM . Lorna Shore . Upcoming Events: 2. The Tabernacle - GA . Sep 25, 2024 6:30 PM ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002Bon\u002BSeptember\u002B25%2c\u002B2024\u0026d=4854189676169673\u0026mkt=en-US\u0026setlang=en-US\u0026w=73iRaVHIvJefbkmdL1lCvH4mS1bjztOY"},{"name":"Atlanta Events - September 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html","snippet":"More details. \u2013 Dragon Con, August 29 - September 2 2024. One of the biggest annual pop culture conventions in the country. More details. \u2013 Atlanta Black Theatre Festival, August 30 - September 1 2024. Annual festival celebrating the Black experience in the performing arts, with plays, panels, masterclasses and other events.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002Bon\u002BSeptember\u002B25%2c\u002B2024\u0026d=4712253893574850\u0026mkt=en-US\u0026setlang=en-US\u0026w=aWkJWqw243Je4oOvzq1dGpeg_GE4sHjt","thumbnailUrl":"https://www.bing.com/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Cool Things To Do in September in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta","snippet":"Published August 23, 2024 Last Updated September 11, 2024. It\u2019s September. In Atlanta that means it\u2019s time for festivals, costumed aliens walking down Peachtree Street and food \u2014 glorious food. Food Extravaganza Take a big bite. There is plenty to sample at the Atlanta Food \u0026amp; Wine Festival with more than 200 chefs, sommeliers and mixologists.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002Bon\u002BSeptember\u002B25%2c\u002B2024\u0026d=4692574352310340\u0026mkt=en-US\u0026setlang=en-US\u0026w=c-5Oucqqe-FV74GkIkdoRE4k4vk9MNbJ","thumbnailUrl":"https://www.bing.com/th?id=OIP.m0ENCio7ytS2o9PQRWDCRgHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Explore Atlanta Events for Today, This Week or Weekend","url":"https://discoveratlanta.com/events/main/","displayUrl":"https://discoveratlanta.com/events/m","snippet":"Explore the Atlanta events calendar for things to do today, this week or this weekend. Find upcoming festivals, seasonal events and more! ... Every Wednesday until 09/25/2024 \u201CMix It Up\u201D Ladies Night ... September 25. Wednesday. BEAT: Steve Vai, Adrian Belew, Tony Levin, Danny Carey 9/25/24. September 25.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002Bon\u002BSeptember\u002B25%2c\u002B2024\u0026d=4819881473496114\u0026mkt=en-US\u0026setlang=en-US\u0026w=zvJBiG0Dg4eP1GMx_bJv9FTM8ziERJGY","thumbnailUrl":"https://www.bing.com/th?id=OIP.FkxSV6JbrWatsFMM3_8dIAHaEh\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"20 Spectacular Things To Do This September In Atlanta - Secret Atlanta","url":"https://secretatlanta.co/things-to-do-september/","displayUrl":"https://secretatlanta.co/things-to-do-september","snippet":"When: Saturday, September 7th from 12pm \u2013 10pm. Thousands of people are expected to head to Morris Brown College in Atlanta this September for the legendary Seafood \u0026amp; Caribbean Music festival! Enjoy tons of live music, DJs, and of course, as much mouthwatering seafood and Caribbean dishes as your heart desires! Don\u2019t miss out on all the fun.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002Bon\u002BSeptember\u002B25%2c\u002B2024\u0026d=4724958405090999\u0026mkt=en-US\u0026setlang=en-US\u0026w=HkmWmkX0emVkqeBkOazQ7cCeg5xgwMQ3","thumbnailUrl":"https://www.bing.com/th?id=OIP.mwxBnPRtbggmsDGaNhGACQHaE8\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Festival Calendar: 2024","url":"https://unexpectedatlanta.com/atlanta-festival-calendar-2024/","displayUrl":"https://unexpectedatlanta.com/atlanta-festival-calendar-2024","snippet":"Atlanta Jazz Festival \u2013 May 25-27, 2024. Held in Piedmont Park, the Atlanta Jazz Festival is an annual celebration of music, culture, and the art of Jazz. Free to all who join, this Memorial Day Weekend event draws fans, professional musicians, and everyone in between, for a 3-day-long festival showcasing 15\u002B artists.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002Bon\u002BSeptember\u002B25%2c\u002B2024\u0026d=4520195830737555\u0026mkt=en-US\u0026setlang=en-US\u0026w=Y_UYHSuQi3QHjG8exIRi2zfYmaaF9rTA","thumbnailUrl":"https://www.bing.com/th?id=OIP.8LvXI6tOuWvo9KZdL-C3MwHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Concerts in September 2024 - Atlanta Theaters","url":"https://atlanta.theater/concerts/september/","displayUrl":"https://atlanta.theater/concerts/september","snippet":"Find the Best Atlanta Concerts in September 2024. Buy Concert Tickets for Upcoming Live Music Events at The Lowest Possible Prices. ... 25. 2024. 6:30 PM Wed. Sevendust. The Eastern - GA. 30312, 800 Old Flat Shoals Road SE, Atlanta, GA, US Prices from $49 Avg. price ~ $89 ... Join our newsletter now to stay current on all theater events in ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002Bon\u002BSeptember\u002B25%2c\u002B2024\u0026d=4792058675200737\u0026mkt=en-US\u0026setlang=en-US\u0026w=nJ4Dnuj1fTiDLRkFwto6a5mCZC_EJTj4"},{"name":"Concerts In Atlanta, September 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-concerts-september.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-concerts-september.html","snippet":"Calendar of upcoming concerts in Atlanta in September 2024: rock, hip hop, pop, country, metal, classical, free concerts, tribute shows, and more. ... \u2013 The Airborne Toxic Event at Variety Playhouse. More details \u2013 Faye Webster at the Fox ... Concerts On Wednesday September 25 2024 \u2013 Creed \u2013 Summer of \u201999 Tour at the Ameris Bank ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002Bon\u002BSeptember\u002B25%2c\u002B2024\u0026d=4997727481785510\u0026mkt=en-US\u0026setlang=en-US\u0026w=7wIlOPp8iibgtBrujIlQ6WBS_NbfP6S4"},{"name":"Atlanta in 2024 | Concerts, Festivals, Tickets \u0026amp; Tour Dates - Songkick","url":"https://www.songkick.com/metro-areas/4120-us-atlanta/2024","displayUrl":"https://www.songkick.com/metro-areas/4120-us-atlanta/2024","snippet":"Wednesday 04 September 2024 \u2013 Saturday 07 September 2024. ProgPower 2024 Dark Tranquillity and Amorphis. Center Stage, Atlanta, GA, US. ProgPower USA XXIII 2024 ILLUMISHADE and VICTORIUS. Center Stage Theater, Atlanta, GA, US. Thursday 05 September 2024. Outdoor. Lynyrd Skynyrd, ZZ Top, and The Outlaws.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002Bon\u002BSeptember\u002B25%2c\u002B2024\u0026d=4943520698868250\u0026mkt=en-US\u0026setlang=en-US\u0026w=Jvl_4hTY7ZLIDd8Wa8glqjNaLOrIMgoZ"},{"name":"REVOLT WORLD - We Are Hip Hop - Sept. 20-22, 2024 - Atlanta, GA","url":"https://www.revoltworld.com/","displayUrl":"https://www.revoltworld.com","snippet":"REVOLT WORLD 2024. REVOLT WORLD returns to Atlanta, GA, from September 20th \u2013 22nd as a first-of-its-kind event reimagining the intersection of culture and creativity. This 3-day immersive experience features top REVOLT shows, impactful conversations, live performances, masterclasses, and branded activations.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002Bon\u002BSeptember\u002B25%2c\u002B2024\u0026d=4622596448205542\u0026mkt=en-US\u0026setlang=en-US\u0026w=PPqa2U-tHIo2HVW4kqdqSY5evF_MRtAT","thumbnailUrl":"https://www.bing.com/th?id=OIP.cZbqheqMyaxKhPF_BVZeIQHaJ4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"}]}}
```

I called the tool named ExtractContentFromUrl with these arguments:
```
{"assistantName":"WebPageAgentAnt","instructions":"What are the events happening in Atlanta on September 25, 2024? https://atlanta-ga.events/september/"}
```
and got this result:
```
"On September 25, 2024, several events are scheduled to take place in Atlanta:\n\n1. **Lorna Shore**\n   - **Venue:** The Tabernacle\n   - **Time:** 6:30 PM\n   - **Capacity:** 2600\n   - **Prices from:** $20\n   - **Average price:** ~$89\n   - **Available Tickets:** 81\n\n2. **Sevendust**\n   - **Venue:** The Eastern\n   - **Time:** 6:30 PM\n   - **Capacity:** 30312\n   - **Prices from:** $39\n   - **Average price:** ~$61\n   - **Available Tickets:** 21\n\n3. **Creed, 3 Doors Down \u0026 Finger Eleven**\n   - **Venue:** Ameris Bank Amphitheatre\n   - **Time:** 7:00 PM\n   - **Capacity:** 12000\n   - **Prices from:** $39\n   - **Average price:** ~$223\n   - **Available Tickets:** 223\n\n4. **Atlanta Braves vs. New York Mets**\n   - **Venue:** Truist Park\n   - **Time:** 7:20 PM\n   - **Capacity:** 41149\n   - **Prices from:** $7\n   - **Average price:** ~$75\n   - **Available Tickets:** 1977\n\n5. **Kings of Leon \u0026 Phantogram**\n   - **Venue:** State Farm Arena\n   - **Time:** 7:30 PM\n   - **Capacity:** 21000\n   - **Prices from:** $14\n   - **Average price:** ~$123\n   - **Available Tickets:** 804\n\n6. **Shaboozey**\n   - **Venue:** Variety Playhouse\n   - **Time:** 8:00 PM\n   - **Capacity:** 1000\n   - **Prices from:** $51\n   - **Average price:** ~$88\n   - **Available Tickets:** 28\n\n7. **Lasso - Artist**\n   - **Venue:** Buckhead Theatre\n   - **Time:** 8:00 PM\n   - **Capacity:** 2500\n   - **Prices from:** $58\n   - **Average price:** ~$90\n   - **Available Tickets:** 54\n\n8. **Gritz and Jelly Butter**\n   - **Venue:** City Winery - Atlanta\n   - **Time:** 8:00 PM\n   - **Capacity:** 300\n   - **Prices from:** $74\n   - **Average price:** ~$89\n   - **Available Tickets:** 8\n\nThese events offer a range of concerts and a sports game, providing various options for entertainment on that day."
```

Here are some events happening in Atlanta on September 25, 2024:

1. **Lorna Shore**
   - **Venue:** The Tabernacle
   - **Time:** 6:30 PM
   - **Prices from:** $20
   - **Average price:** ~$89
   - **Available Tickets:** 81

2. **Sevendust**
   - **Venue:** The Eastern
   - **Time:** 6:30 PM
   - **Prices from:** $39
   - **Average price:** ~$61
   - **Available Tickets:** 21

3. **Creed, 3 Doors Down & Finger Eleven**
   - **Venue:** Ameris Bank Amphitheatre
   - **Time:** 7:00 PM
   - **Prices from:** $39
   - **Average price:** ~$223
   - **Available Tickets:** 223

4. **Atlanta Braves vs. New York Mets**
   - **Venue:** Truist Park
   - **Time:** 7:20 PM
   - **Prices from:** $7
   - **Average price:** ~$75
   - **Available Tickets:** 1977

5. **Kings of Leon & Phantogram**
   - **Venue:** State Farm Arena
   - **Time:** 7:30 PM
   - **Prices from:** $14
   - **Average price:** ~$123
   - **Available Tickets:** 804

6. **Shaboozey**
   - **Venue:** Variety Playhouse
   - **Time:** 8:00 PM
   - **Prices from:** $51
   - **Average price:** ~$88
   - **Available Tickets:** 28

7. **Lasso - Artist**
   - **Venue:** Buckhead Theatre
   - **Time:** 8:00 PM
   - **Prices from:** $58
   - **Average price:** ~$90
   - **Available Tickets:** 54

8. **Gritz and Jelly Butter**
   - **Venue:** City Winery - Atlanta
   - **Time:** 8:00 PM
   - **Prices from:** $74
   - **Average price:** ~$89
   - **Available Tickets:** 8

These events cover a variety of interests from concerts to sports, offering a range of entertainment options for that day.

For more details, you can visit the source: [Atlanta Events September 2024](https://atlanta-ga.events/september/).


In [12]:
output.LastMessage.DisplayAs("text/markdown");

Here are some events happening in Atlanta on September 25, 2024:

1. **Lorna Shore**
   - **Venue:** The Tabernacle
   - **Time:** 6:30 PM
   - **Prices from:** $20
   - **Average price:** ~$89
   - **Available Tickets:** 81

2. **Sevendust**
   - **Venue:** The Eastern
   - **Time:** 6:30 PM
   - **Prices from:** $39
   - **Average price:** ~$61
   - **Available Tickets:** 21

3. **Creed, 3 Doors Down & Finger Eleven**
   - **Venue:** Ameris Bank Amphitheatre
   - **Time:** 7:00 PM
   - **Prices from:** $39
   - **Average price:** ~$223
   - **Available Tickets:** 223

4. **Atlanta Braves vs. New York Mets**
   - **Venue:** Truist Park
   - **Time:** 7:20 PM
   - **Prices from:** $7
   - **Average price:** ~$75
   - **Available Tickets:** 1977

5. **Kings of Leon & Phantogram**
   - **Venue:** State Farm Arena
   - **Time:** 7:30 PM
   - **Prices from:** $14
   - **Average price:** ~$123
   - **Available Tickets:** 804

6. **Shaboozey**
   - **Venue:** Variety Playhouse
   - **Time:** 8:00 PM
   - **Prices from:** $51
   - **Average price:** ~$88
   - **Available Tickets:** 28

7. **Lasso - Artist**
   - **Venue:** Buckhead Theatre
   - **Time:** 8:00 PM
   - **Prices from:** $58
   - **Average price:** ~$90
   - **Available Tickets:** 54

8. **Gritz and Jelly Butter**
   - **Venue:** City Winery - Atlanta
   - **Time:** 8:00 PM
   - **Prices from:** $74
   - **Average price:** ~$89
   - **Available Tickets:** 8

These events cover a variety of interests from concerts to sports, offering a range of entertainment options for that day.

For more details, you can visit the source: [Atlanta Events September 2024](https://atlanta-ga.events/september/).

In [13]:
var output2 = await AntRunnerLib.AssistantRunner.RunThread("WebSearchAgentAnt", "Use python to get todays date and then use it to find things to do in Atlanta for each of the next four weekends with pictures and **correct links**.\n\nWhen choosing links for crawl to evaluate with ExtractContentFromUrl choose a variety of hosts for urls from which you extract content. Limit your use of ExtractContentFromUrl to five calls or less. Do not make the error of including the url as a parameter, it must be inside 'instructions' as indicated by the tool definition. Be careful to not include events in the past in your final answer. You should perform four searches at once.\nDon't forget to include valid links!");
output2.DisplayAs("text/markdown");

Here are some exciting things to do in Atlanta for each of the next four weekends:

### September 28-29, 2024
1. **Sweet Auburn Music Fest**: This festival, located on Auburn Avenue, offers entertainment, live music ranging from R&B to gospel to alternative favorites, an international food court, national artists, celebrities, and various vendors.
2. **Riverfest**: An annual juried arts and crafts festival in Etowah River Park, Canton, featuring over 150 artists and artisans.
3. **East Atlanta Strut**: This iconic street festival celebrates East Atlanta’s culture, art, and quirkiness with activities and entertainment.

![Sweet Auburn Music Fest](https://www.bing.com/th?id=OIP.m0ENCio7ytS2o9PQRWDCRgHaD4&w=80&h=80&c=1&pid=5.1)

For more details, visit: [Event Cartel](https://eventcartel.com/events/all-categories/atlanta/from_2024-09-28_to_2024-09-29)

### October 5-6, 2024
1. **Festival on Ponce**: A twice-annual arts festival in Olmsted Linear Park with over 125 vendors of fine art, folk art, and outsider art, plus food and drink, and children’s activities.
2. **Candler Park Fall Fest**: This popular street festival includes an artist market, live acoustic music on two stages, rides and special attractions, a tour of homes, food, kids activities, and more.
3. **Atlanta Fair**: Bringing the best of a traditional fair to Atlanta with cotton candy, a Ferris wheel, fishing for goldfish, a tilt-a-whirl, and a carousel.

![Festival on Ponce](https://www.bing.com/th?id=OIP.4jf9xqKu3QRk_XhZ3J7DYQHaD4&w=80&h=80&c=1&pid=5.1)

For more details, visit: [Go South Atlanta](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html)

### October 12-13, 2024
1. **FashionFest**: This immersive festival combines elements of a festival and an exhibition, showcasing urban and pop culture. Attendees can engage with brands, enjoy live music, explore a curated marketplace, and learn about business principles in the fashion industry.
2. **Art Exhibitions**: The exhibition "If You Look Hard Enough, You Can See Our Future" at the African Diaspora Art Museum of Atlanta features works by African artists and runs through November 2.
3. **Culinary Experiences**: Many restaurants in Atlanta offer unique brunch options during the fall, perfect for enjoying the seasonal flavors.

For more details, visit: [Discover Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta)

### October 19-20, 2024
1. **Little Five Points Halloween Festival and Parade**: This festival features live music from local underground acts, an art and vintage market, ghost tours, a skate park, and food and drink vendors. The parade is known as the biggest and spookiest in the Southeast and has been recognized as one of the best Halloween parades in the country.
2. **Atlanta Streets Alive**: On October 20, the streets will be closed to vehicular traffic, allowing walkers, skaters, and cyclists to enjoy Ralph David Abernathy Boulevard and Georgia Avenue from 1 PM to 5 PM.

![Little Five Points Halloween Festival](https://res.cloudinary.com/atlanta/images/w_2560,h_1707,c_scale/v1726166287/newAtlanta.com/23-Halloween-Reid-Koski_4-copy/23-Halloween-Reid-Koski_4-copy.jpg?_i=AA)

For more details, visit: [Discover Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta)

Enjoy your weekends in Atlanta!

## Clean Up

In [14]:
var assistant = (await client.AssistantList())?.Data?.FirstOrDefault(o => o.Name == "WebSearchAgentAnt");
if(assistant != null) {
    await client.AssistantDelete(assistant.Id);
    Console.WriteLine("Deleted assistant");
}
else
{
    Console.WriteLine("Didn't find WebSearchAgentAnt");
}

assistant = (await client.AssistantList())?.Data?.FirstOrDefault(o => o.Name == "WebPageAgentAnt");
if(assistant != null) {
    await client.AssistantDelete(assistant.Id);
    Console.WriteLine("Deleted assistant");
}
else
{
    Console.WriteLine("Didn't find WebSearchAgentAnt");
}

Deleted assistant
Deleted assistant
